In [2]:
from pyspark.sql import SparkSession
import os
import configparser
import pyspark.sql.functions as F
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType, StringType

# Start spark session

In [3]:
spark = SparkSession.builder\
                     .config("spark.jars.packages","org.apache.hadoop:hadoop-aws:2.7.0")\
                     .getOrCreate()

# Read data from Parquet

In [32]:
df_artists = spark.read.parquet("OUT/ARTISTS")
df_songs = spark.read.parquet("OUT/SONGS")
df_users = spark.read.parquet("OUT/USERS/")
df_timestamps = spark.read.parquet("OUT/TIMESTAMPS/")
df_songplays = spark.read.parquet("OUT/SONGPLAYS/")

## Number of rows per table

In [5]:
print("artists :", df_artists.count())
print("songs :", df_songs.count())
print("users :", df_users.count())
print("timestamps :", df_timestamps.count())
print("songplays :", df_songplays.count())

artists : 69
songs : 71
users : 104
timestamps : 6813
songplays : 4


## Database schemas

In [6]:
print("Artists schema :")
df_artists.printSchema()
print("Songs schema :")
df_songs.printSchema()
print("Users schema :")
df_users.printSchema()
print("Timestamps schema :")
df_timestamps.printSchema()
print("Songplays schema :")
df_songplays.printSchema()

Artists schema :
root
 |-- artist_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- location: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)

Songs schema :
root
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- year: integer (nullable = true)
 |-- artist_id: string (nullable = true)

Users schema :
root
 |-- user_id: long (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- level: string (nullable = true)

Timestamps schema :
root
 |-- ts: long (nullable = true)
 |-- hour: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- week: integer (nullable = true)
 |-- weekday: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)

Songplays schema :
root
 |-- songplay_id: long (nullable = true)


# Example queries

## Artists with most songs

In [7]:
df_songs.join(df_artists, df_songs["artist_id"] == df_artists["artist_id"])\
    .groupBy(df_artists["artist_id"], "name")\
    .count()\
    .orderBy("count", "name", ascending = [False, True])\
    .select(col("name").alias("artist name"), "count")\
    .limit(10)\
    .toPandas()


,artist name,count
0,Casual,2
1,Clp,2
2,40 Grit,1
3,Adam Ant,1
4,Alice Stuart,1
5,Andy Andy,1
6,Backstreet Boys,1
7,Billie Jo Spears,1
8,Bitter End,1
9,Blingtones,1


## Biggest song consumers

In [8]:
df_songplays.join(df_users, df_songplays["user_id"] == df_users["user_id"])\
    .groupBy(df_users["user_id"], "first_name", "last_name")\
    .count()\
    .orderBy(col("count").desc(), col("last_name").asc() )\
    .select("first_name", "last_name", "count")\
    .limit(10)\
    .toPandas()

,first_name,last_name,count
0,Lily,Koch,2
1,Tegan,Levine,2
2,Sylvie,Cruz,1
3,Layla,Griffin,1


## Average usage per week day

In [9]:
import calendar
list(calendar.day_name)

['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

In [10]:
@udf(returnType= StringType())
def weekdaynumToName(daynum):
    '''
    convert the day of week number (0 to 6) to name ("Monday" -> "Sunday")
    '''
    return calendar.day_name[daynum]

In [ ]:
# drop columns year and months from songplays (duplicate from timestamps)
# and join on ts (start time of songplay)
join_plays_timestamps = df_songplays.withColumnRenamed("start_time", "ts")\
        .drop("year", "month")\
        .join(df_timestamps, ["ts"])
join_plays_timestamps.printSchema()

In [37]:
# nb of distinct dates for a given weekday in the database
num_weeks = join_plays_timestamps \
        .select("weekday", "year", "month", "day")\
        .groupBy("weekday")\
        .agg( F.countDistinct("year", "month","day"))\
        .orderBy("weekday")\
        .select(weekdaynumToName("weekday").alias("week day"))\
        .toPandas()
    
join_plays_timestamps.groupBy(df_timestamps["weekday"])\
    .count()\
    .orderBy("weekday")\
    .select(weekdaynumToName("weekday").alias("week day"), "count")\
    .toPandas()
    

,week day,count
0,Wednesday,1
1,Thursday,1
2,Friday,2


## Percentage of songs in the database that the users listen

In [18]:
nb_songs = df_songs.count()
nb_songplays = df_songplays.select("song_id").distinct().count()
print("Percentage of songs in the database which are listened to :", 
      100.*nb_songplays/nb_songs)

Percentage of songs in the database which are listened to : 2.816901408450704
